# QQ图分析：分泌蛋白 vs 非分泌蛋白的Ssec评分比较

## 分析目的

本notebook用于生成**QQ图（Quantile-Quantile Plot）**，比较分泌蛋白和非分泌蛋白的Ssec显著性评分分布，从而验证QENIE方法能够有效识别内分泌互作相关的分泌蛋白。

## 生物学假设

如果QENIE方法有效，我们期望：
1. **分泌蛋白**的Ssec评分应当显著高于非分泌蛋白
2. 因为只有分泌蛋白能够通过血液循环在组织间传递信号
3. 非分泌蛋白即使与目标组织基因相关，也不太可能是真正的内分泌因子

## QQ图原理

### 什么是QQ图？

**QQ图（Quantile-Quantile Plot）**是一种图形化方法，用于比较两个概率分布：

- **X轴**：非分泌蛋白的Ssec评分分位数
- **Y轴**：分泌蛋白的Ssec评分分位数
- **对角线**（红色）：表示两个分布完全相同

### 如何解读QQ图？

1. **点在对角线上**：两组数据分布相同
   - 说明分泌蛋白和非分泌蛋白的Ssec评分无差异
   - 方法可能无效

2. **点在对角线上方**：分泌蛋白评分更高
   - Y值 > X值
   - 说明相同分位数下，分泌蛋白的Ssec评分更高
   - **这是我们期望的结果**
   - 证明方法能够富集分泌蛋白

3. **点在对角线下方**：非分泌蛋白评分更高
   - Y值 < X值
   - 说明方法可能有问题

### 分位数标注

图中会标注几个关键分位数（如1%、10%、50%、90%、99%）：
- **低分位数**（如1%、10%）：评分较低的基因
- **中分位数**（如50%）：中等评分的基因
- **高分位数**（如90%、99%）：评分较高的基因（最感兴趣的候选者）

如果高分位数的点明显偏离对角线上方，说明**排名靠前的基因中，分泌蛋白的比例显著高于随机预期**。

## 统计学意义

QQ图提供了一种直观的方法来评估：
1. **富集效果**：分泌蛋白在高评分基因中的富集程度
2. **方法效度**：QENIE能否区分真正的内分泌因子
3. **阈值选择**：帮助确定合适的Ssec评分阈值

## 第一步：环境准备与数据导入

### 所需R包

- **WGCNA**：计算跨组织相关性p值
  - bicorAndPvalue函数：返回相关系数和p值
  
### 脚本来源说明

本脚本由**Simon Koplev**设计和提供，用于生成QQ图可视化。

特别感谢**IIT Madras的BIRDS实验室**的Arjun Sarathi和Manikandan Narayanan，他们通过详细的troubleshooting发现并修正了原始QQ图坐标轴的错误。

In [ ]:
# 加载WGCNA包
# 如果未安装，请运行：install.packages("WGCNA")
library(WGCNA)

In [ ]:
# 导入分泌蛋白列表
# 这个列表来自UniProt数据库，包含所有小鼠分泌蛋白
Secreted_proteins <- read.delim("Secreted_proteins_Uniprot.txt", header = T, check.names = F)

print("分泌蛋白列表加载完成")
print(paste("分泌蛋白总数：", nrow(Secreted_proteins)))

In [ ]:
# 导入脂肪和肝脏组织表达数据
# 这些数据来自GEO数据库GSE64770
# 包含106个HMDP小鼠品系的12,242个基因表达值

Adipose <- read.delim('adipose.txt', check.names=F)
Liver <- read.delim('liver.txt', check.names=F)

print("表达数据加载完成")
print(paste("脂肪组织数据维度：", nrow(Adipose), "×", ncol(Adipose)))
print(paste("肝脏组织数据维度：", nrow(Liver), "×", ncol(Liver)))

## 第二步：计算跨组织相关性P值

### 计算内容

本步骤计算肝脏和脂肪组织间所有基因对的相关性p值。

### 与主分析的区别

在主分析notebook中，我们计算了相关系数和p值：
```R
liv.adip = bicorAndPvalue(Liver, Adipose, use='pairwise.complete.obs')
# 使用了 liv.adip$bicor (相关系数) 和 liv.adip$p (p值)
```

在QQ图分析中，我们只需要p值：
```R
liv.adip.p = bicorAndPvalue(Liver, Adipose, use='pairwise.complete.obs')$p
# 直接提取p值矩阵
```

### 为什么只用P值？

- QQ图只需要比较Ssec评分（基于p值计算）的分布
- 不需要相关系数的方向信息
- 简化计算，节省内存

### 计算Ssec评分

对每个肝脏基因：
```
Ssec = Σ(-log(p值))  # 对该基因与所有脂肪基因的p值求和
```

**注意**：为了专注于QQ图比较，这里不进行归一化处理。

In [ ]:
# 计算跨组织相关性的p值矩阵
# 这一步可能需要几分钟时间

print("开始计算跨组织相关性p值...")
print("这可能需要几分钟时间，请耐心等待...")

# 计算双权重中值相关性的p值
# $p: 直接提取p值矩阵，不保存相关系数
liv.adip.p = bicorAndPvalue(Liver, Adipose, use='pairwise.complete.obs')$p

print("\n计算完成！")
print(paste("p值矩阵维度：", nrow(liv.adip.p), "×", ncol(liv.adip.p)))

In [ ]:
# 计算每个基因的Ssec评分
# rowSums: 对每一行（每个肝脏基因）求和
# -log(p): p值越小，-log(p)越大

print("计算Ssec显著性评分...")
scores = rowSums(-log(liv.adip.p))

print("\nSsec评分统计：")
print(paste("基因总数：", length(scores)))
print(paste("最高分：", round(max(scores), 2)))
print(paste("最低分：", round(min(scores), 2)))
print(paste("平均分：", round(mean(scores), 2)))
print(paste("中位数：", round(median(scores), 2)))
print(paste("标准差：", round(sd(scores), 2)))

## 第三步：分离分泌蛋白和非分泌蛋白的评分

### 分组策略

将所有基因分为两组：

1. **分泌蛋白组（sec）**
   - 基因名在UniProt分泌蛋白列表中
   - 这些蛋白能够被分泌到细胞外
   - 理论上更可能是内分泌因子

2. **非分泌蛋白组（non_sec）**
   - 基因名不在分泌蛋白列表中
   - 这些蛋白主要在细胞内发挥作用
   - 作为对照组

### R语法说明

```R
# %in% 操作符：检查左侧元素是否在右侧向量中
names(scores) %in% Secreted_proteins$`Gene names  (primary )`
# 返回逻辑向量：TRUE（在列表中）或FALSE（不在列表中）

# 方括号索引：使用逻辑向量筛选
sec = scores[逻辑向量]  # 提取TRUE位置的元素

# 感叹号(!)：逻辑取反
!逻辑向量  # 将TRUE变FALSE，FALSE变TRUE
```

### 生物学解释

通过这种分组，我们可以回答：
- 分泌蛋白的Ssec评分是否系统性地高于非分泌蛋白？
- 如果是，说明QENIE能够从基因表达相关性中识别出真正的内分泌因子

In [ ]:
# 将基因分为分泌蛋白和非分泌蛋白两组

print("分离分泌蛋白和非分泌蛋白的评分...")

# 提取分泌蛋白的评分
# %in%: 检查基因名是否在分泌蛋白列表中
sec = scores[names(scores) %in% Secreted_proteins$`Gene names  (primary )`]

# 提取非分泌蛋白的评分
# !: 逻辑取反，选择不在分泌蛋白列表中的基因
non_sec = scores[!(names(scores) %in% Secreted_proteins$`Gene names  (primary )`)]

In [ ]:
# 比较两组的统计特征

print("========== 分泌蛋白组 ==========")
print(paste("数量：", length(sec)))
print(paste("最高分：", round(max(sec), 2)))
print(paste("最低分：", round(min(sec), 2)))
print(paste("平均分：", round(mean(sec), 2)))
print(paste("中位数：", round(median(sec), 2)))
print(paste("标准差：", round(sd(sec), 2)))

print("\n========== 非分泌蛋白组 ==========")
print(paste("数量：", length(non_sec)))
print(paste("最高分：", round(max(non_sec), 2)))
print(paste("最低分：", round(min(non_sec), 2)))
print(paste("平均分：", round(mean(non_sec), 2)))
print(paste("中位数：", round(median(non_sec), 2)))
print(paste("标准差：", round(sd(non_sec), 2)))

print("\n========== 比较 ==========")
print(paste("平均分差异：", round(mean(sec) - mean(non_sec), 2)))
print(paste("中位数差异：", round(median(sec) - median(non_sec), 2)))

# 进行t检验评估两组差异的显著性
t_test_result = t.test(sec, non_sec)
print(paste("\nt检验p值：", format(t_test_result$p.value, scientific = TRUE)))
if(t_test_result$p.value < 0.05) {
  print("结论：分泌蛋白的Ssec评分显著高于非分泌蛋白（p < 0.05）")
} else {
  print("结论：两组无显著差异（p ≥ 0.05）")
}

## 第四步：定义增强版QQ图绘制函数

### 函数功能

**qqplotAnnot**函数是标准qqplot的增强版本，增加了：
1. 参考对角线（红色）
2. 关键分位数标注（灰色点和标签）
3. 更清晰的可视化效果

### 参数说明

- **x**: 第一组数据（非分泌蛋白的评分）
  - 对应X轴
  - 作为参考分布
  
- **y**: 第二组数据（分泌蛋白的评分）
  - 对应Y轴
  - 要比较的分布
  
- **probs**: 要标注的分位数
  - 默认：0.001, 0.01, 0.1, 0.9, 0.99, 0.999
  - 对应0.1%, 1%, 10%, 90%, 99%, 99.9%分位数
  - 覆盖了从低到高的全范围
  
- **...**: 传递给qqplot的其他参数
  - main: 图标题
  - xlab: X轴标签
  - ylab: Y轴标签

### 绘图步骤

1. **绘制基础QQ图**
   ```R
   qqplot(x, y, pch=16, cex=0.5, ...)
   ```
   - pch=16: 实心圆点
   - cex=0.5: 点的大小为默认的50%

2. **添加参考对角线**
   ```R
   abline(0, 1, col="red")
   ```
   - 截距为0，斜率为1
   - 表示两个分布完全相同的情况
   - 红色便于识别

3. **计算并标注关键分位数**
   ```R
   q_y = quantile(y, probs=probs, na.rm=TRUE)  # Y轴分位数
   q_x = quantile(x, probs=probs, na.rm=TRUE)  # X轴分位数
   points(q_x, q_y, col="grey", cex=0.6)       # 灰色点标记
   text(q_x, q_y, label=..., pos=1, ...)       # 百分比标签
   ```

### 可视化元素

- **黑色小点**：所有基因的分位数对
- **红色对角线**：两组分布相同的参考线
- **灰色点**：关键分位数位置
- **百分比标签**：如"1%", "10%", "90%", "99%"

### 解读技巧

- **整体偏移**：如果大部分点都在对角线上方，说明分泌蛋白整体评分更高
- **尾部分离**：如果高分位数（90%、99%）的点明显偏离对角线，说明顶级候选者富集了分泌蛋白
- **线性关系**：点的分布是否呈线性，反映两组分布形状的相似性

In [ ]:
# 定义带分位数标注的QQ图绘制函数
# 这个函数由Simon Koplev设计

qqplotAnnot = function(x, y,
                       probs=c(0.001, 0.01, 0.1, 0.9, 0.99, 0.999),
                       ...)
{
  # 步骤1: 绘制基础QQ图
  # qqplot自动对两组数据排序并配对相应的分位数
  qqplot(
    x,              # X轴数据（非分泌蛋白评分）
    y,              # Y轴数据（分泌蛋白评分）
    pch=16,         # 点的形状：实心圆
    cex=0.5,        # 点的大小：默认的50%
    ...             # 其他参数（如标题、轴标签等）
  )
  
  # 步骤2: 添加参考对角线
  # 如果两组分布相同，所有点应落在这条线上
  abline(0, 1, col="red")  # 截距0，斜率1，红色

  # 步骤3: 计算指定的分位数
  # quantile函数：计算数据的分位数
  # na.rm=TRUE: 忽略缺失值
  q_y = quantile(y, probs=probs, na.rm=TRUE)  # Y轴（分泌蛋白）的分位数
  q_x = quantile(x, probs=probs, na.rm=TRUE)  # X轴（非分泌蛋白）的分位数

  # 步骤4: 在关键分位数位置添加标记点
  points(q_x, q_y, col="grey", cex=0.6)
  
  # 步骤5: 添加百分比标签
  # paste0: 字符串拼接，无分隔符
  # pos=1: 标签位置在点的下方
  text(q_x, q_y, 
       label=paste0(probs*100, "%"),  # 将概率转为百分比
       pos=1,                           # 标签在点下方
       cex=0.8,                        # 字体大小
       col="grey")                     # 灰色字体
}

print("QQ图绘制函数定义完成")

## 第五步：生成并解读QQ图

### 图形元素说明

1. **标题**: "QQ-plot, Liver x adipose"
   - 说明这是肝脏-脂肪组织回路的QQ图

2. **X轴**: "Non-secreted factor Ssec (-ln p)"
   - 非分泌蛋白的Ssec评分
   - -ln p = -log(p)
   - 作为参考分布

3. **Y轴**: "Secreted factor Ssec (-ln p)"
   - 分泌蛋白的Ssec评分
   - 要评估的目标分布

### 期望结果

如果QENIE方法有效，我们应该看到：

1. **大部分点在红色对角线上方**
   - Y值 > X值
   - 说明分泌蛋白的Ssec评分普遍高于非分泌蛋白

2. **高分位数（90%、99%、99.9%）显著偏离对角线**
   - 表明排名靠前的基因中，分泌蛋白显著富集
   - 这是最重要的特征，因为我们主要关注高评分的候选基因

3. **偏离程度递增**
   - 随着分位数增加，偏离对角线的距离增大
   - 说明评分越高，分泌蛋白的富集越明显

### 解读示例

假设99%分位数的点坐标为(100, 150)：
- X轴（非分泌蛋白）：在99%分位数处，评分为100
- Y轴（分泌蛋白）：在99%分位数处，评分为150
- **解读**：前1%的分泌蛋白评分（150）远高于前1%的非分泌蛋白（100）
- **生物学意义**：顶级候选内分泌因子确实主要是分泌蛋白

### 不理想的结果

如果出现以下情况，可能需要重新审视方法或数据：
1. **点落在对角线上或下方**：无富集或反向富集
2. **高分位数无明显偏离**：方法对高评分基因的区分能力不足
3. **分布极度非线性**：两组数据的分布形状差异过大，可能有数据质量问题

In [ ]:
# 设置图形输出参数
# 确保图形有足够的分辨率和合适的尺寸
options(repr.plot.width=10, repr.plot.height=10)  # Jupyter中的图形尺寸

# 绘制QQ图
print("生成QQ图...")

qqplotAnnot(non_sec,                              # X轴：非分泌蛋白评分
            sec,                                   # Y轴：分泌蛋白评分
            main = "QQ-plot, Liver x adipose",    # 图标题
            xlab = "Non-secreted factor Ssec (-ln p)",  # X轴标签
            ylab = "Secreted factor Ssec (-ln p)"        # Y轴标签
)

print("\nQQ图生成完成！")

## 结果解读指南

### 快速检查清单

查看生成的QQ图，检查以下几点：

#### ✓ 预期的良好结果

- [ ] 大部分点在红色对角线上方
- [ ] 高分位数（90%、99%、99.9%）明显偏离对角线向上
- [ ] 低分位数（0.1%、1%、10%）接近对角线
- [ ] 整体呈现"弓形"向上的趋势

#### ✗ 需要注意的情况

- [ ] 点落在对角线下方：方法可能有问题
- [ ] 所有点都在对角线上：无富集效果
- [ ] 极度非线性：数据质量可能有问题
- [ ] 高分位数无明显分离：方法对候选基因区分能力弱

### 定量评估

除了视觉观察，可以进行定量评估：

In [ ]:
# 计算关键分位数的富集倍数
print("========== 分位数富集分析 ==========")
print("\n计算不同分位数下分泌蛋白相对于非分泌蛋白的富集倍数：\n")

key_probs = c(0.5, 0.9, 0.95, 0.99, 0.999)

for(prob in key_probs) {
  # 计算该分位数下两组的评分值
  q_sec = quantile(sec, probs=prob, na.rm=TRUE)
  q_nonsec = quantile(non_sec, probs=prob, na.rm=TRUE)
  
  # 计算富集倍数
  enrichment = q_sec / q_nonsec
  
  # 计算差值
  difference = q_sec - q_nonsec
  
  print(paste0(prob*100, "% 分位数："))
  print(paste("  分泌蛋白评分：", round(q_sec, 2)))
  print(paste("  非分泌蛋白评分：", round(q_nonsec, 2)))
  print(paste("  富集倍数：", round(enrichment, 2), "倍"))
  print(paste("  差值：", round(difference, 2)))
  print("")
}

print("\n解读说明：")
print("- 富集倍数 > 1：分泌蛋白评分更高（期望结果）")
print("- 富集倍数越大：分泌蛋白富集越明显")
print("- 高分位数的富集倍数应该更大，说明顶级候选者中分泌蛋白富集")

## 统计检验

### Kolmogorov-Smirnov检验

除了t检验，我们还可以使用**K-S检验**来比较两个分布的整体差异：

- **零假设**：两个分布相同
- **备择假设**：两个分布不同
- **优势**：非参数检验，不假设数据服从正态分布
- **D统计量**：两个累积分布函数之间的最大距离
  - D越大，分布差异越大
  - 范围：0到1

In [ ]:
# 进行Kolmogorov-Smirnov检验
print("========== Kolmogorov-Smirnov检验 ==========")

ks_result = ks.test(sec, non_sec)

print(paste("D统计量：", round(ks_result$statistic, 4)))
print(paste("p值：", format(ks_result$p.value, scientific = TRUE)))

if(ks_result$p.value < 0.05) {
  print("\n结论：分泌蛋白和非分泌蛋白的Ssec评分分布显著不同（p < 0.05）")
  print("这支持QENIE方法能够有效区分分泌蛋白和非分泌蛋白的假设。")
} else {
  print("\n结论：两组分布无显著差异（p ≥ 0.05）")
  print("这可能表明方法需要改进或数据存在问题。")
}

print("\n\n========== Wilcoxon秩和检验 ==========")
# 另一种非参数检验，比较两组的中位数
wilcox_result = wilcox.test(sec, non_sec)

print(paste("W统计量：", round(wilcox_result$statistic, 2)))
print(paste("p值：", format(wilcox_result$p.value, scientific = TRUE)))

if(wilcox_result$p.value < 0.05) {
  print("\n结论：分泌蛋白的Ssec评分中位数显著高于非分泌蛋白（p < 0.05）")
} else {
  print("\n结论：两组中位数无显著差异（p ≥ 0.05）")
}

## 补充可视化：直方图和箱线图

除了QQ图，我们还可以用其他方式可视化两组数据的差异：

In [ ]:
# 绘制重叠的直方图
print("生成Ssec评分分布的直方图...")

# 设置图形参数
par(mfrow=c(1,1))  # 1行1列的图形布局

# 确定合适的x轴范围和分箱数
xlim_range = range(c(sec, non_sec))
breaks_num = 50

# 绘制非分泌蛋白的直方图（灰色，半透明）
hist(non_sec, 
     breaks=breaks_num,
     col=rgb(0.5, 0.5, 0.5, 0.5),  # 灰色，透明度50%
     xlim=xlim_range,
     main="Ssec评分分布比较",
     xlab="Ssec评分 (-ln p)",
     ylab="基因数量",
     border="white")

# 叠加分泌蛋白的直方图（红色，半透明）
hist(sec, 
     breaks=breaks_num,
     col=rgb(1, 0, 0, 0.5),  # 红色，透明度50%
     add=TRUE,                # 添加到现有图上
     border="white")

# 添加图例
legend("topright",
       legend=c("非分泌蛋白", "分泌蛋白"),
       fill=c(rgb(0.5, 0.5, 0.5, 0.5), rgb(1, 0, 0, 0.5)),
       border="black")

# 添加均值线
abline(v=mean(non_sec), col="grey", lwd=2, lty=2)  # 非分泌蛋白均值
abline(v=mean(sec), col="red", lwd=2, lty=2)       # 分泌蛋白均值

print("直方图生成完成！")
print("图中虚线表示各组的平均值")

In [ ]:
# 绘制箱线图比较
print("生成箱线图比较...")

# 准备数据框
boxplot_data = data.frame(
  score = c(non_sec, sec),
  group = c(rep("非分泌蛋白", length(non_sec)), 
            rep("分泌蛋白", length(sec)))
)

# 绘制箱线图
boxplot(score ~ group, 
        data = boxplot_data,
        col = c("grey", "red"),
        main = "Ssec评分箱线图比较",
        ylab = "Ssec评分 (-ln p)",
        xlab = "蛋白类型",
        notch = TRUE,     # 添加凹槽，用于比较中位数
        outline = FALSE)  # 不显示异常值，避免图形过于拥挤

# 添加均值点
points(c(1, 2), 
       c(mean(non_sec), mean(sec)),
       col = "blue",
       pch = 18,  # 菱形
       cex = 2)   # 点的大小

legend("topright",
       legend = c("中位数（凹槽）", "平均值（蓝色菱形）"),
       pch = c(NA, 18),
       col = c(NA, "blue"),
       lty = c(1, NA),
       bty = "n")  # 无边框

print("箱线图生成完成！")
print("\n箱线图解读：")
print("- 箱体：四分位距（IQR），包含中间50%的数据")
print("- 中线：中位数")
print("- 凹槽：中位数的置信区间，如果两组凹槽不重叠，说明中位数有显著差异")
print("- 蓝色菱形：平均值")

## 分析总结

### QQ图的价值

QQ图提供了一种直观、强大的方法来评估QENIE的有效性：

1. **可视化优势**
   - 一目了然地展示两组分布的差异
   - 容易识别系统性偏移
   - 可以看到整个分布范围的比较

2. **生物学验证**
   - 如果高分位数显著偏离对角线，说明：
     - QENIE能够识别真正的内分泌因子
     - 高Ssec评分的基因富集了分泌蛋白
     - 方法具有生物学有效性

3. **方法改进**
   - 如果结果不理想，可以考虑：
     - 调整相关性计算方法
     - 改变p值转换方式
     - 使用不同的归一化策略
     - 优化数据预处理步骤

### 与主分析的关系

| 分析类型 | 主分析Notebook | QQ图Notebook |
|---------|---------------|-------------|
| **目的** | 识别候选内分泌因子 | 验证方法有效性 |
| **输出** | 排序的基因列表 | 可视化评估 |
| **关注点** | 具体基因和通路 | 整体分布特征 |
| **使用时机** | 每次新分析 | 方法验证和发表 |

### 推荐工作流程

1. **首次使用QENIE时**：
   - 先运行QQ图分析，验证方法在你的数据上是否有效
   - 如果QQ图显示良好的分泌蛋白富集，继续使用方法
   - 如果QQ图不理想，检查数据质量或调整参数

2. **常规分析时**：
   - 使用主分析流程识别候选基因
   - 进行通路富集分析
   - 实验验证候选基因

3. **发表研究时**：
   - 在论文的方法验证部分包含QQ图
   - 报告统计检验结果（t检验、K-S检验）
   - 展示分位数富集倍数

### 引用致谢

如果您在研究中使用了这个QQ图分析方法，请考虑致谢：

- **Simon Koplev**：设计并提供QQ图脚本
- **IIT Madras BIRDS实验室**：
  - Arjun Sarathi
  - Manikandan (Mani) Narayanan
  - 发现并修正QQ图坐标轴错误

### 进一步阅读

关于QQ图的更多信息：
- Wilk, M. B., & Gnanadesikan, R. (1968). "Probability plotting methods for the analysis of data". Biometrika.
- 在线教程：Wikipedia "Q-Q plot"

关于WGCNA和bicor相关性：
- Langfelder, P., & Horvath, S. (2008). "WGCNA: an R package for weighted correlation network analysis". BMC Bioinformatics.
- WGCNA官网：https://horvath.genetics.ucla.edu/html/CoexpressionNetwork/Rpackages/WGCNA/